## Open Babel python interface

In [ ]:
from openbabel import OBMol, OBConversion, OBElementTable
from openbabel import OBMolAtomIter, OBAtomBondIter, OBForceField

### File conversion

In [ ]:
obconversion = OBConversion()
obconversion.SetInAndOutFormats("xyz", "pdb")  # Set input and output formats

In [ ]:
obmol = OBMol()                                # Create openbabel molecule instance
mol_file = 'benzene.xyz'                       # Select input file
obconversion.ReadFile(obmol, mol_file)         # Read file (file is read into obmol object)
obconversion.WriteFile(obmol, 'benzene.pdb')   # Convert file to output format and save

### Periodic table

In [ ]:
table = OBElementTable()             # Setup element table

element = 'C'
num = table.GetAtomicNum(element)    # Get atomic number from element symbol
mass = table.GetMass(num)            # Get mass from atomic number 
symbol = table.GetSymbol(num)        # Get element symbol from atomic number
name = table.GetName(num)            # Get element name
r_cov = table.GetCovalentRad(num)    # Get covalent radius
r_vdw = table.GetVdwRad(num)         # Get van der Waals radius
en = table.GetElectroNeg(num)        # Get electronegativity 

print('Element: %s | Symbol: %s | No: %i | Mass: %.3f | EN: %.2f | Radius cov: %.2f vdw: %.2f'
      % (name, symbol, num, mass, en, r_cov, r_vdw))

### Topology

In [ ]:
bond_ids = []
for obatom in OBMolAtomIter(obmol):                                    # Iterate over atoms
    for bond in OBAtomBondIter(obatom):                                # Iterate over bonds
        atom1 = table.GetSymbol(bond.GetBeginAtom().GetAtomicNum())    # Get symbol for bonded atom 1
        atom2 = table.GetSymbol(bond.GetEndAtom().GetAtomicNum())      # Get symbol for bonded atom 2
        atom1_idx = bond.GetBeginAtomIdx()                             # Atom 1 index
        atom2_idx = bond.GetEndAtomIdx()                               # Atom 2 index
        order = bond.GetBondOrder()                                    # Bond order
        length = bond.GetLength()                                      # Bond length
        eq_length = bond.GetEquibLength()                              # Equilibrium bond length
        bond_id = bond.GetIdx()                                        # Bond id
        if bond_id not in bond_ids:
            bond_ids.append(bond.GetIdx())   
            print('%s %2i - %s %2i | Order: %i | Length: actual-> %.3f - equilibrium-> %.3f'
                  % (atom1, atom1_idx, atom2, atom2_idx, order, length, eq_length))

### Force field

In [ ]:
ff = OBForceField.FindForceField("UFF")               # Select force field
if not ff.Setup(obmol):                               # Set force field for molecule
    print("Error: could not setup force field")       # Print this message if cannot set force field

ff.GetAtomTypes(obmol)                                # Get force field atom types

In [ ]:
for obatom in OBMolAtomIter(obmol):
    num = obatom.GetAtomicNum()                                  # Atomic number
    element = table.GetSymbol(num)                               # Element symbol
    x, y, z = obatom.GetX(), obatom.GetY(), obatom.GetZ()        # Coordinates 
    ff_atom_type = obatom.GetData("FFAtomType")                  # Force field atom type
    ff_atom_type = ff_atom_type.GetValue()                       # ...
    valence = obatom.GetValence()                                # Valence
    print('%-7s %-2i %-5s %-i %6.3f %6.3f %6.3f' % (element, num, ff_atom_type, valence, x, y, z))